In [326]:
from random import randint
from math import exp

In [327]:
learning_rate = 3
epoch = 100

imput_values = [[1,1,1],[1,0,1],[0,1,1],[0,0,1]]
out_expected_values = [0, 1, 1, 0]


neuron_hidden_layer_1 = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
neuron_hidden_layer_2 = [[0,0],[0,0],[0,0],[0,0]]
out_actual_values = [0,0,0,0]

neuron_hidden_layer_error_1 = [
    [[0,0],[0,0],[0,0]],
    [[0,0],[0,0],[0,0]],
    [[0,0],[0,0],[0,0]],
    [[0,0],[0,0],[0,0]]
    ]
neuron_hidden_layer_error_2 = [[0,0],[0,0],[0,0],[0,0]]

weights_1 = [
    [randint(0,1000)/1000,randint(0,1000)/1000,randint(0,1000)/1000],
    [randint(0,1000)/1000,randint(0,1000)/1000,randint(0,1000)/1000],
    [randint(0,1000)/1000,randint(0,1000)/1000,randint(0,1000)/1000]
    ]

weights_2 = [
    [randint(0,1000)/1000,randint(0,1000)/1000],
    [randint(0,1000)/1000,randint(0,1000)/1000]
    ]

weights_3 = [randint(0,1000)/1000,randint(0,1000)/1000]

In [328]:
def sigmoid(x):
    return 1 / (1 + exp(-x))

In [329]:
def calculation_first_error(actual, expected):
    return actual - expected

def calculation_error(weight, delta):
    return weight * delta

In [330]:
def calculation_delta(value, error):
    weights_delta = error * sigmoid(value) * (1-sigmoid(value))
    return weights_delta

In [331]:
def correct_weight(weight, value, delta, learning_rate):
    return weight - value * delta * learning_rate

In [332]:
def backpropagation_error(
    neuron_hidden_layer_1,
    neuron_hidden_layer_2,
    out_actual_values,
    out_expected_values,
    weights_1,
    weights_2,
    weights_3,
    learning_rate,
    neuron_hidden_layer_error_1,
    neuron_hidden_layer_error_2,
    imput_values):

    


    for i in range(len(out_actual_values)):
        first_error = calculation_first_error(out_actual_values[i], out_expected_values[i])
        for j in range(len(neuron_hidden_layer_2[0])):
            
            delta = calculation_delta(neuron_hidden_layer_2[i][j], first_error)
            weights_3[j] = correct_weight(weights_3[j], neuron_hidden_layer_2[i][j], delta, learning_rate)

            neuron_hidden_layer_error_2[i][j] = calculation_error(weights_3[j], delta)
    

    for i in range(len(neuron_hidden_layer_2)):
        for j in range(len(neuron_hidden_layer_1[0])):
            for k in range(len(neuron_hidden_layer_error_2[0])):
                for l in range(len(weights_2)):
                    
                    delta = calculation_delta(neuron_hidden_layer_1[i][j], neuron_hidden_layer_error_2[i][l])
                    weights_2[k][l] = correct_weight(weights_2[k][l], neuron_hidden_layer_1[i][j], delta, learning_rate)

                    neuron_hidden_layer_error_1[i][j][k] = calculation_error(weights_2[k][l], delta)


    for i in range(len(neuron_hidden_layer_1)):
        for j in range(len(imput_values[0])):
            for k in range(len(neuron_hidden_layer_error_1[0][0])):

                delta = calculation_delta(imput_values[i][j], neuron_hidden_layer_error_1[i][j][k])

                for l in range(len(weights_1[0])):
                
                    weights_1[j][l] = correct_weight(weights_1[j][l], neuron_hidden_layer_1[i][j], delta, learning_rate)
    
    return weights_1, weights_2, weights_2

In [333]:
def running(imput_values, weights_1, weights_2, neuron_hidden_layer_1, neuron_hidden_layer_2,out_actual_values):
    
    for i in range(len(imput_values)):
        for j in range(len(neuron_hidden_layer_1[0])):
            summator_1 = 0
            for l in range(len(weights_1)):
                summator_1 += imput_values[i][l] * weights_1[l][j]
            neuron_hidden_layer_1[i][j] = sigmoid(summator_1)

    for i in range(len(neuron_hidden_layer_1)):
        for j in range(len(neuron_hidden_layer_2[0])):
            summator_2 = 0
            for l in range(len(weights_2)):
                summator_2 += neuron_hidden_layer_1[i][l] * weights_2[l][j]
            neuron_hidden_layer_2[i][j] = sigmoid(summator_2)
        
    for i in range(len(neuron_hidden_layer_2)):
        summator_3 = 0
        for l in range(len(weights_3)):
            summator_3 += neuron_hidden_layer_2[i][l] * weights_3[l]
        out_actual_values[i] = sigmoid(summator_3)

    return neuron_hidden_layer_1, neuron_hidden_layer_2, out_actual_values

In [334]:
def train(
    epoch,
    neuron_hidden_layer_1,
    neuron_hidden_layer_2,
    out_actual_values,
    out_expected_values,
    weights_1,
    weights_2,
    weights_3,
    learning_rate,
    neuron_hidden_layer_error_1,
    neuron_hidden_layer_error_2,
    imput_values):
    
    for i in range(epoch):
        neuron_hidden_layer_1, neuron_hidden_layer_2, out_actual_values = running(imput_values, weights_1, weights_2, neuron_hidden_layer_1, neuron_hidden_layer_2, out_actual_values)
        print(out_actual_values)
        weights_1, weights_2, weights_2 = backpropagation_error(
            neuron_hidden_layer_1,
            neuron_hidden_layer_2,
            out_actual_values,
            out_expected_values,
            weights_1,
            weights_2,
            weights_3,
            learning_rate,
            neuron_hidden_layer_error_1,
            neuron_hidden_layer_error_2,
            imput_values)

In [335]:
train(
    epoch,
    neuron_hidden_layer_1,
    neuron_hidden_layer_2,
    out_actual_values,
    out_expected_values,
    weights_1,
    weights_2,
    weights_3,
    learning_rate,
    neuron_hidden_layer_error_1,
    neuron_hidden_layer_error_2,
    imput_values)

[0.726352933485333, 0.7238137674247128, 0.7241495459289018, 0.7214114083756109]
[0.5712254892636571, 0.5703338730670547, 0.5705986377090928, 0.5696643553933198]
[0.5195930458005502, 0.5193023979280962, 0.519497766902772, 0.519205620382011]
[0.5050343309299743, 0.504903260045776, 0.5050706996696429, 0.5049513521630747]
[0.501480750809477, 0.5013746339104379, 0.5015262183845101, 0.5014353651262552]
[0.5007597516501106, 0.5006435769211566, 0.5007816854466578, 0.5006807616693657]
[0.5006572545377784, 0.5005241515535148, 0.5006490521862353, 0.5005295815797148]
[0.5006644305646353, 0.5005145795926098, 0.5006263147538981, 0.5004873496658397]
[0.5006843648282574, 0.500519744361174, 0.5006184955129661, 0.5004609511024833]
[0.5007019487728385, 0.5005252195064045, 0.5006113557914524, 0.5004368380205722]
[0.5007150225144176, 0.5005292627974021, 0.5006033336225979, 0.5004139201231015]
[0.5007230203688576, 0.5005315796193344, 0.5005942945307285, 0.5003924576137011]
[0.5007255818877221, 0.50053193211